In [ ]:
#instalando o selenium
!pip install selenium

In [1]:
#imports
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from pprint import pprint
from selenium import webdriver
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.relative_locator import locate_with
import os

In [144]:
#link da página de certificações da RIAA filtrado de 01/01/2010 a 31/12/2019
URL = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar=&ti=&lab=&genre=&format=&date_option=release&from=2010-01-01&to=2019-12-31&award=&type=&category=&adv=SEARCH#search_section"

In [ ]:
#usando o driver do selenium para abrir e maximizar a página em questão
PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

driver.get(URL)
driver.maximize_window()

#criando um bot que carrega a página completa, clicando no botão "Load More" e depois exibe mais informações de cada certificação, clicando no botão "MORE DETAILS"
THERES_MORE = True

while THERES_MORE:

	WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@class='footer-content']")))

	try:
		driver.find_element_by_id('loadmore').click()
		time.sleep(1)
	except:
		THERES_MORE = False
        
like = driver.find_elements_by_link_text('MORE DETAILS')
for x in range(0,len(like)):
    if like[x].is_displayed():
        like[x].click()
        time.sleep(1)

#salvando o código fonte completo da página para possíveis consultas        
pageSource = driver.page_source
fileToWrite = open("page_source.html", "w")
fileToWrite.write(pageSource)
fileToWrite.close()
fileToRead = open("page_source.html", "r")
print(fileToRead.read())

In [ ]:
#o tipo de certificação ("Gold", "Platinum", "Multi Platinum", "Diamond") é definido pela imagem exibida na faixa de cada certificação
#dessa forma, precisamos selecionar todas as imagens e filtrar pela URL das imagens que correspondem à certificações

#selecionando todas as imagens da página
img = driver.find_elements_by_tag_name('img')
links = []
for image in img:
    links.append(image.get_attribute('src'))
    #print(image.get_attribute('src'))
len(links)

In [ ]:
#selecionando as imagens que representam uma certificação
links_imgs = []
for i in links:
    if "https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/" and "_big.png" in i:
        links_imgs.append(i)

links_imgs.pop(0)
print(links_imgs)
len(links_imgs)

In [ ]:
#a página teve bugs em 24 certificações, por esse motivo excluiremos essas informações das próximas tabelas

#selecionando a parcela do link que corresponde à certificação
award_links = []
for link in range(0,13986,2):
    award_links.append(links_imgs[link])
award_links = [i.replace("https://www.riaa.com/wp-content/plugins/riaa-gnp/img/icons/", "") for i in award_links]
award_links = [i.replace("_big.png", "") for i in award_links]
award_links.pop(6297)
award_links.pop(3359)
award_links.pop(28)
award_links.pop(27)
award_links.pop(25)
award_links.pop(23)
award_links.pop(22)
award_links.pop(21)
award_links.pop(19)
award_links.pop(17)
award_links.pop(16)
award_links.pop(15)
award_links.pop(14)
award_links.pop(13)
award_links.pop(12)
award_links.pop(11)
award_links.pop(8)
award_links.pop(7)
award_links.pop(6)
award_links.pop(5)
award_links.pop(4)
award_links.pop(2)
award_links.pop(1)
print(award_links)
len(award_links)

In [ ]:
#retirando a especificação de certificação latina "la_" na string
new_award_links = []
for i in award_links:
    if "la_" in i:
        i = i.replace("la_", "")
        new_award_links.append(i)
    else:
        new_award_links.append(i)
        
print(new_award_links)
len(new_award_links)

In [ ]:
#baseado no valor proveninente do link especificaremos cada uma das certificações
#0 = gold
#1 = platinum
#de 2 a 9 = multi platinum
#10 ou mais = diamond
awards = []
for i in new_award_links:
    if int(i) == 0:
        awards.append("Gold")
    if int(i) == 1:
        awards.append("Platinum")
    if 2 <= int(i) < 10:
        awards.append("Multi platinum")
    if int(i) >= 10 :
        awards.append("Diamond")
        
print(awards)   
len(awards)

In [ ]:
awards.pop(6360)
len(awards)

In [ ]:
#selecionando todos os nomes de artistas
artist = driver.find_elements_by_class_name("artists_cell")
x1 = 0
artista = []
for artist in artist:
    artista.append(artist.text)
    x1 = x1+1
print(x1)

In [ ]:
artista.pop(6297)
artista.pop(3359)
artista.pop(28)
artista.pop(27)
artista.pop(25)
artista.pop(23)
artista.pop(22)
artista.pop(21)
artista.pop(19)
artista.pop(17)
artista.pop(16)
artista.pop(15)
artista.pop(14)
artista.pop(13)
artista.pop(12)
artista.pop(11)
artista.pop(8)
artista.pop(7)
artista.pop(6)
artista.pop(5)
artista.pop(4)
artista.pop(2)
artista.pop(1)

print(artista)
len(artista)

In [ ]:
artista.pop(6360)
len(artista)

In [ ]:
#selecionando maiores informações da faixa da certificação
infos = driver.find_elements_by_class_name("others_cell")
y1 = 0
for info in infos:
    #print(info.text)
    y1 = y1+1
print(y1)

In [ ]:
#retirando das informações os títulos das faixas
titles = []
for title in range(0,27972,4):
    titles.append(infos[title].text)
titles.pop(6297)
titles.pop(3359)
titles.pop(28)
titles.pop(27)
titles.pop(25)
titles.pop(23)
titles.pop(22)
titles.pop(21)
titles.pop(19)
titles.pop(17)
titles.pop(16)
titles.pop(15)
titles.pop(14)
titles.pop(13)
titles.pop(12)
titles.pop(11)
titles.pop(8)
titles.pop(7)
titles.pop(6)
titles.pop(5)
titles.pop(4)
titles.pop(2)
titles.pop(1)
print(titles)
len(titles)

In [ ]:
titles.pop(6360)
len(titles)

In [ ]:
#retirando das informações as datas de certificação
certifdate = []
for date in range(1,27972,4):
    certifdate.append(infos[date].text)
certifdate.pop(6297)
certifdate.pop(3359)
certifdate.pop(28)
certifdate.pop(27)
certifdate.pop(25)
certifdate.pop(23)
certifdate.pop(22)
certifdate.pop(21)
certifdate.pop(19)
certifdate.pop(17)
certifdate.pop(16)
certifdate.pop(15)
certifdate.pop(14)
certifdate.pop(13)
certifdate.pop(12)
certifdate.pop(11)
certifdate.pop(8)
certifdate.pop(7)
certifdate.pop(6)
certifdate.pop(5)
certifdate.pop(4)
certifdate.pop(2)
certifdate.pop(1)
print(certifdate)
len(certifdate)

In [ ]:
certifdate.pop(6360)
len(certifdate)

In [ ]:
#retirando das informações as gravadoras de cada faixa
label = []
for label in range(2,27972,4):
    label.append(infos[label].text)
label.pop(6297)
label.pop(3359)
label.pop(28)
label.pop(27)
label.pop(25)
label.pop(23)
label.pop(22)
label.pop(21)
label.pop(19)
label.pop(17)
label.pop(16)
label.pop(15)
label.pop(14)
label.pop(13)
label.pop(12)
label.pop(11)
label.pop(8)
label.pop(7)
label.pop(6)
label.pop(5)
label.pop(4)
label.pop(2)
label.pop(1)
print(label)
len(label)

In [ ]:
label.pop(6360)
len(label)

In [ ]:
#retirando das informações o formato da faixa (single, album, video longform, etc)
format = []
for formato in range(3,27972,4):
    format.append(infos[formato].text)
format.pop(6297)
format.pop(3359)
format.pop(28)
format.pop(27)
format.pop(25)
format.pop(23)
format.pop(22)
format.pop(21)
format.pop(19)
format.pop(17)
format.pop(16)
format.pop(15)
format.pop(14)
format.pop(13)
format.pop(12)
format.pop(11)
format.pop(8)
format.pop(7)
format.pop(6)
format.pop(5)
format.pop(4)
format.pop(2)
format.pop(1)
format = [i.replace("\nMORE DETAILS", "") for i in format]
print(format)
len(format)

In [ ]:
format.pop(6360)
len(format)

In [ ]:
#selecionando todas as datas de lançamento (de jan/2010 a dez/2019)
release = driver.find_elements_by_class_name("upper_style")
a1 = 0
for release in release:
    #print(release.text)
    a1 = a1+1
print(a1)

In [ ]:
for release in release:
    print(release.text)
len(release)

In [ ]:
releasedate = []
for date in range(0,6969):
    releasedate.append(release[date].text)
print(releasedate)
len(releasedate)

In [ ]:
#selecionando todas as informações de unidades certificadas
certif_units = driver.find_elements_by_class_name("col-md-4")
c1 = 0
for certif in certif_units:
    #print(certif.text)
    c1 = c1+1
print(c1)

In [ ]:
units = []
for unit in certif_units:
    if "Million" in unit.text:
        units.append(unit.text)
print(units)
len(units)

In [331]:
#criando o dataframe com as listas
dados_certification = {"Certificação" : awards,
         "Artista(s)" : artista, 
         "Título" : titles,
         "Formato" : format,
         "Data de Lançamento" : releasedate,
         "Data de Certificação" : certifdate,
         "Gravadora(s)" : label,
         "Unidades Certificadas" : units
         }
df_certification = pd.DataFrame(dados_certification)
df_certification

,Certificação,Artista(s),Título,Formato,Data de Lançamento,Data de Certificação,Gravadora(s),Unidades Certificadas
0,Multi platinum,NIKO MOON,GOOD TIME,SINGLE,"JULY 19, 2019","November 22, 2021",RCA NASHVILLE,2 Million
1,Multi platinum,BROCKHAMPTON,SUGAR,SINGLE,"AUGUST 23, 2019","November 19, 2021",QUESTION EVERYTHING / RCA RECORDS,2 Million
2,Platinum,HOUNDMOUTH,SEDONA,SINGLE,"JANUARY 20, 2015","November 17, 2021",ROUGH TRADE,1 Million
3,Platinum,CASTING CROWNS,NOBODY (FEAT. MATTHEW WEST),SINGLE,"OCTOBER 12, 2018","November 16, 2021",BEACH STREET RECORDS / REUNION RECORDS,1 Million
4,Platinum,YOUNG M.A,BIG,SINGLE,"JUNE 28, 2019","November 12, 2021",M.A MUSIC / 3D,1 Million
...,...,...,...,...,...,...,...,...
6964,Gold,MONICA,STILL STANDING,ALBUM,"MARCH 23, 2010","April 21, 2010",J RECORDS,0.5 Million
6965,Platinum,LUDACRIS,HOW LOW,SINGLE,"MARCH 9, 2010","April 9, 2010",DEF JAM,1 Million
6966,Platinum,BANDA LOS RECODITOS,ANDO BIEN PEDO,ALBUM,"JANUARY 19, 2010","March 19, 2010",DISA,0.06 Million
6967,Platinum,SADE,SOLDIER OF LOVE,ALBUM,"FEBRUARY 9, 2010","March 15, 2010",COLUMBIA,1 Million


In [332]:
#salvando .csv com o dataframe
df_certification.to_csv('tabela-certification.csv')